<a href="https://colab.research.google.com/github/REICHIYAN/REI/blob/master/Dog%26Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from icrawler.builtin import BingImageCrawler

crawler = BingImageCrawler(storage={"root_dir": "cats"})
crawler.crawl(keyword="猫", max_num=100)

2020-05-19 15:01:39,387 - INFO - icrawler.crawler - start crawling...
2020-05-19 15:01:39,388 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-05-19 15:01:39,393 - INFO - feeder - thread feeder-001 exit
2020-05-19 15:01:39,393 - INFO - icrawler.crawler - starting 1 parser threads...
2020-05-19 15:01:39,396 - INFO - icrawler.crawler - starting 1 downloader threads...
2020-05-19 15:01:39,778 - INFO - parser - parsing result page https://www.bing.com/images/async?q=猫&first=0
2020-05-19 15:01:40,608 - INFO - downloader - image #1	https://public.potaufeu.asahi.com/686b-p/picture/12463073/5c4a362cea9cb2f5d90b60e2f2a6c85f.jpg
2020-05-19 15:01:40,832 - INFO - downloader - image #2	https://imgcp.aacdn.jp/img-a/1200/900/aa/gm/article/3/1/8/0/4/201802071158/topimg_original.jpg
2020-05-19 15:01:40,856 - INFO - downloader - image #3	https://i.ytimg.com/vi/k3J5pHsh8Y4/hqdefault.jpg
2020-05-19 15:01:40,959 - INFO - downloader - image #4	https://ddnavi.com/wp-content/uploads/2018/07/suzum

In [0]:
from icrawler.builtin import BingImageCrawler

crawler = BingImageCrawler(storage={"root_dir": "dogs"})
crawler.crawl(keyword="犬", max_num=100)

2020-05-19 15:04:14,320 - INFO - icrawler.crawler - start crawling...
2020-05-19 15:04:14,321 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-05-19 15:04:14,323 - INFO - feeder - thread feeder-001 exit
2020-05-19 15:04:14,323 - INFO - icrawler.crawler - starting 1 parser threads...
2020-05-19 15:04:14,326 - INFO - icrawler.crawler - starting 1 downloader threads...
2020-05-19 15:04:14,634 - INFO - parser - parsing result page https://www.bing.com/images/async?q=犬&first=0
2020-05-19 15:04:14,711 - INFO - downloader - image #1	https://i.ytimg.com/vi/7bTKGFiYil0/maxresdefault.jpg
2020-05-19 15:04:14,794 - INFO - downloader - image #2	https://i.ytimg.com/vi/pflWZcX9Tr4/maxresdefault.jpg
2020-05-19 15:04:14,847 - INFO - downloader - image #3	https://i.ytimg.com/vi/Oml9D2j-KGA/maxresdefault.jpg
2020-05-19 15:04:15,714 - INFO - downloader - image #4	https://cdn0.mynvwm.com/wp-content/uploads/2019/02/adorable-animal-breed-1805164-e1549450482765.jpg
2020-05-19 15:04:16,079 - INFO 

In [0]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["dogs", "cats"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                
                img_trans = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trans)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./dog_cat.npy", xy)

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64

def load_data():
    X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)

    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test


def train(X, y, X_test, y_test):
    model = Sequential()

    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = keras.optimizers.rmsprop(lr=0.00005, decay=1e-6)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)

    model.save('./cnn.h5')

    return model


def main():

    X_train, y_train, X_test, y_test = load_data()

    model = train(X_train, y_train, X_test, y_test)

main()

Using TensorFlow backend.


Epoch 1/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.6728 - accuracy: 0.5794
Epoch 2/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.6242 - accuracy: 0.6488
Epoch 3/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.5583 - accuracy: 0.7050
Epoch 4/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.5228 - accuracy: 0.7352
Epoch 5/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.4865 - accuracy: 0.7564
Epoch 6/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.4581 - accuracy: 0.7722
Epoch 7/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.4290 - accuracy: 0.7807
Epoch 8/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.4035 - accuracy: 0.7987
Epoch 9/40
1888/1888 [==============================] - 36s 19ms/step - loss: 0.3834 - accuracy: 0.8257
Epoch 10/40
1888/1888 [==============================] - 36s 19m

In [0]:
import keras
import sys, os
import numpy as np
from keras.models import load_model

imsize = (64, 64)


testpic     = "./cat1.jpg"
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')

    img = img.resize(imsize)

    img = np.asarray(img)
    img = img / 255.0
    return img

if __name__ == "__main__":

    model = load_model(keras_param)
    img = load_image(testpic)
    prd = model.predict(np.array([img]))
    print(prd)
    prelabel = np.argmax(prd, axis=1)
    if prelabel == 0:
        print(">>> 犬")
    elif prelabel == 1:
        print(">>> 猫")

[[0.31368777 0.6863122 ]]
>>> 猫
